# Technique - Recursion (Backtracking)

Backtracking is a recursive technique that can be used for exhaustive searching.

## Example [Restore IP Addresses](https://leetcode.com/problems/restore-ip-addresses/submissions/)

This problem is an exhaustive search - we need to find all possible IP addresses from a given string of ints. We can solve it with backtracking, or with straightforward recursion.

### Identifying a recurrence
A valid IPv4 address is of the form `x.x.x.x` where `0 <= x 255`, and x has no leading `0` (unless `x == 0`). Let's say that we have three types of IPs:
- A valid IP address, e.g. `24.123.54.4`
- An invalid IP address:
    - `525.1.2.4` (field exceeds 255)
    - `01.1.24.5` (field has leading zero)
    - `1..3.4` (empty field)
    - `1.2.3.4.5` (too many fields)
- An incomplete IP address, e.g. an IP that is only invalid because it has too few fields:
    - `1.2.3` 
    - `1.2`
    - `1.`
    - `(empty string)`

For any IP address `s` and a character `x` s.t. `0 <= x <= 9`, we can either try adding `x` as the final character of s, or appending `x` to the end of `s` as a new field:
- If `s` is valid:
    - We cannot append `x` as a new field, e.g. if `s = 1.2.3.4`, `1.2.3.4.x` is not valid. 
    - We can possibly appending `x` as a final character: `1.2.3.4x` is valid.
- If `s` is invalid, there is no way to append `x` to produce a valid or incomplete IP. 
- If `s` is incomplete:
    - Appending `x` as a field produces either a valid IP (`s = 1.2.3, x = 4, s.x = 1.2.3.4`) or another incomplete IP (`s = 1.2, x = 3, s.x = 1.2.3`)
    - Appending `x` as a final character produces an incomplete IP (`s = 1.2.1, x = 4, sx = 1.2.14`) or an invalid IP (`s = 1.2.99, x = 1, sx = 1.2.991`). 
    
So given an input `string` of int characters, our recurrence relationship is:
```
all valid IPs ending at string[i] =
    any valid result of appending s[i] as a character to all valid IPs ending at string[i-1],    
    any valid result of appending s[i] as a character to all incomplete IPs ending at string[i-1],
    any valid result of appending s[i] as a field to all incomplete IPs ending at string[i-1]
```

## Base case and recursive case
Because the problem says to use all characters in `string`, our base case will be: `current IP address is invalid, or current IP address is valid and i == len(string)`. The recursive case is `current IP is incomplete or i < len(string)`, and tries `append string[i] as character, go to i+1` and `append string[i] as a field, go to i+1)`.

In [24]:
from typing import List

def ip_invalid(ip_list):
    if not ip_list:
        return False
    return (len(ip_list) > 4) or \
           not (0 <= int(ip_list[-1]) <= 255) or \
           (ip_list[-1][0] == '0' and len(ip_list[-1]) > 1)
                                    
class Solution:
    def restoreIpAddresses(self, s: str) -> List[str]:
        valid_addrs = []
        
        def find_valid(i, curr):
            if ip_invalid(curr):
                return
            if i >= len(s):
                valid_addrs.append('.'.join(curr)) if len(curr) == 4 else None
                return
            find_valid(i+1, curr + [s[i]])
            find_valid(i+1, curr[:-1] + [curr[-1] + s[i]]) if curr else None

        find_valid(0, [])
        return valid_addrs

sol = Solution()
cases = [
    ("25525511135", ["255.255.11.135","255.255.111.35"]),
    ("0000", ["0.0.0.0"]),
    ("1111", ["1.1.1.1"]),
    ("010010", ["0.10.0.10","0.100.1.0"]),
    ("101023", ["1.0.10.23","1.0.102.3","10.1.0.23","10.10.2.3","101.0.2.3"])   
]
for s, expected in cases:
    actual = sol.restoreIpAddresses(s)
    assert sorted(actual) == sorted(expected), f"{s}: {sorted(expected)} != {sorted(actual)}"